# Predicted Validation Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Validation Data
'''
Loading Validation Data for which Coarse Grained Model 
(NonHostile vs Hostile Classification) predicted Hostile Class
'''
file = '/content/drive/My Drive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Dataset/hostile_validate_pred.xlsx'
test_df = pd.read_excel(file)

# Data Preparation into Pandas Dataframe for Model Input

def get_data(a):
  Unique_ID = list(a['Unique ID'])
  sentence = list(a['Post'])
  text_labels = list(a['Labels Set'])

  label = []
  for i in text_labels:
    if i=='hostile':
        label.append(0)

  raw_data_train = {'UID':Unique_ID,'sentence': sentence, 'label': label}
  df = pd.DataFrame(raw_data_train, columns = ['UID','sentence','label'])
  return df

test_data  = get_data(test_df)

print(test_data[0:3])

# Hostile ID Collection 

In [ ]:
# Collecting Hostile IDs from Validation Data

data = test_data

hos_ids = []
for i in range(len(data)):
  id = data['UID'][i]
  hos_ids.append(id)

hos_ids = np.array(hos_ids, dtype=np.int)
np.save('Predicted_Hostile_ID_Task1.npy',hos_ids)

# Predicted Labels Loading (Fake, Hate, Offensive, Defamation)

In [ ]:
d_lab = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/Final_Defamation_validation_Pred_Label.npy', allow_pickle=True)
f_lab = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/Final_Fake_validation_Pred_Label.npy', allow_pickle=True)
h_lab = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/Final_Hate_validation_Pred_Label.npy', allow_pickle=True)
o_lab = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/Final_Offensive_validation_Pred_Label.npy', allow_pickle=True)
hos_ids = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/Predicted_Hostile_ID_Task1.npy', allow_pickle=True)

# Merging Labels to Asses Performance measurement

In [ ]:
# Merging Predicted labels into a single numpy array
# Reference: [non_hostile,defamation,fake,hate,offensive]

predicted_labels = []

count = 0
for i in range(1,812):
  row = []
  if i not in hos_ids:
    row.append([1,0,0,0,0])
  else:
    alt_row = [0,0,0,0,0]
    if d_lab[count]==1:
      alt_row[1] = 1
    if f_lab[count]==1:
      alt_row[2] = 1
    if h_lab[count]==1:
      alt_row[3] = 1
    if o_lab[count]==1:
      alt_row[4] = 1
    count += 1
    row.append(alt_row)
  predicted_labels.append(row)

pred_lab = np.reshape(np.array(predicted_labels),(811,5)) # Final Predicted Labels
np.save('Final_Pred_Labels_Validation.npy',pred_lab)

# Loading True Labels (811,5) and Predicted Labels (811,5) for Final Result Calculation

In [ ]:
y_true = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/True_Validation_labels.npy', allow_pickle=True)
y_pred = np.load('/content/drive/MyDrive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Final Task Data/Final_Pred_Labels_Validation.npy', allow_pickle=True)

# Computing Scores

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_true, y_pred, average='macro'))
print(f1_score(y_true, y_pred, average='micro'))
print(f1_score(y_true, y_pred, average='weighted'))

# Creating Final Submission File

In [ ]:
# Reference: [non_hostile,defamation,fake,hate,offensive]
labels = []
for i in range(y_pred.shape[0]):
  lab_text = []
  idx = np.argwhere(y_pred[i]>0)
  idx = idx.reshape(idx.shape[0],)
  if idx.shape[0]==0:
    lab_text.append('non_hostile')
  else:
    for j in idx:
      if j==0:
        lab_text.append('non-hostile')
      if j==1:
        lab_text.append('defamation')
      if j==2:
        lab_text.append('fake')
      if j==3:
        lab_text.append('hate')
      if j==4:
        lab_text.append('offensive')
  labels.append(lab_text)

def final_submission(label_list):
  import csv
  data = []
  titles = ['Unique ID','Labels Set']
  data.append(titles)
  for i in range(len(label_list)):
    row = []
    row.append(i+1)
    lab_text = ''
    for j in range(len(label_list[i])):
      lab_text += str(label_list[i][j])+','
    lab_text = lab_text[:-1]+''
    row.append(str(lab_text))
    data.append(row)

  file1 = "answer.csv"
  with open(file1, 'w') as csvfile:  
    csvwriter = csv.writer(csvfile)   
    csvwriter.writerows(data)

final_submission(labels)